In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import scipy.io   
import h5py

import matplotlib.pyplot as plt

import os
import sys
import tarfile
from IPython.display import display, Image

from scipy import ndimage

import random

In [2]:
%matplotlib inline

## Load dataset

In [3]:
pickle_file = 'SVHN_multi_crop.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (27401, 32, 32, 1) (27401, 6)
Validation set (6000, 32, 32, 1) (6000, 6)
Test set (13068, 32, 32, 1) (13068, 6)


In [53]:
batch_size = 128
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden = 1024

image_size = 32
num_labels = 11 # 0-9, + blank 
num_channels = 1 # grayscale

graph = tf.Graph()

with graph.as_default():

    def weight_varible(shape):
        initial = tf.truncated_normal(shape, stddev = 0.1)
        return tf.Variable(initial)

    def bias_varible(shape):
        initial = tf.constant(1.0, shape = shape)
        return tf.Variable(initial)
    
    def conv2d(data, weight):
        # strides [1, x_movement, y_movement, 1]
        return tf.nn.conv2d(data, weight, strides = [1, 1, 1, 1], padding = 'SAME')

    def max_pooling(data):
        return tf.nn.max_pool(data, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
    def get_class_wb():
        weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
        biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
#         logits = tf.matmul(tf_train_dataset, weights) + biases
        return weights, biases #, logits

    
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
    # Varibles
    # conv1 layer 1
    layer1_weights = weight_varible([patch_size, patch_size, num_channels, depth1])
    layer1_biases = bias_varible([depth1]) # 16
    # conv2 layer 2
    layer2_weights = weight_varible([patch_size, patch_size, depth1, depth2]) # in depth1, out depth2
    layer2_biases = bias_varible([depth2]) # 32
    # conv3 layer 3
    layer3_weights = weight_varible([patch_size, patch_size, depth2, depth3]) # in depth2, out depth3
    layer3_biases = bias_varible([depth3]) # 64
    
    
    s1_w, s1_b = get_class_wb()
    s2_w, s2_b = get_class_wb()
    s3_w, s3_b = get_class_wb()
    s4_w, s4_b = get_class_wb()
    s5_w, s5_b = get_class_wb()
    
#     # func1 layer 4
#     layer3_weights = weight_varible([image_size // 8 * image_size // 8 * depth3, num_hidden])
#     layer3_biases = bias_varible([num_hidden])
#     # func2 layer 5
#     layer4_weights = weight_varible([num_hidden, num_labels])
#     layer4_biases = bias_varible([num_labels])
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    def model(dataset):
        
        # conv1 layer 1
        hidden1 = tf.nn.relu(conv2d(dataset, layer1_weights) + layer1_biases) # 32 * 32 * depth1
        pool1 = max_pooling(hidden1) # 16 * 16 * depth1
        # conv2 layer 2
        hidden2 = tf.nn.relu(conv2d(pool1, layer2_weights) + layer2_biases) # 16 * 16 * depth2
        pool2 = max_pooling(hidden2) # 8 * 8 * depth2
        # conv3 layer 3
        hidden3 = tf.nn.relu(conv2d(pool2, layer3_weights) + layer3_biases) # 8 * 8 * depth3
        pool3 = max_pooling(hidden3) # 4 * 4 * depth3
        
        shape = pool3.get_shape().as_list()
        pool3_flat = tf.reshape(pool3, [shape[0], shape[1] * shape[2] * shape[3]])

#         print([shape[0], shape[1] * shape[2] * shape[3]])
#         print([image_size // 4 * image_size // 4 * depth2, num_hidden])

        logits_1 = tf.matmul(pool3_flat, s1_w) + s1_b
        logits_2 = tf.matmul(pool3_flat, s2_w) + s2_b
        logits_3 = tf.matmul(pool3_flat, s3_w) + s3_b
        logits_4 = tf.matmul(pool3_flat, s4_w) + s4_b
        logits_5 = tf.matmul(pool3_flat, s5_w) + s5_b
    
        
#         # func1 layer 4
#         hidden4 = tf.nn.relu(tf.matmul(pool3_flat, layer4_weights) + layer4_biases)
#         hidden4_drop = tf.nn.dropout(hidden3, 0.5)
#         # func2 layer 5
#         prediction = tf.matmul(hidden4_drop, layer5_weights) + layer5_biases
        
        
        return [logits_1, logits_2, logits_3, logits_4, logits_5]
    
# Training computation.
    [logits_1, logits_2, logits_3, logits_4, logits_5] = model(tf_train_dataset)
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_1, tf_train_labels[:,1])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_2, tf_train_labels[:,2])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_3, tf_train_labels[:,3])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_4, tf_train_labels[:,4])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_5, tf_train_labels[:,5]))

#     loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(all_logits, tf_train_labels))
    
    # Optimizer.

    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
#     train_prediction = tf.nn.softmax(logits_1)
#     valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
#     test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.pack([tf.nn.softmax(logits_1),\
                      tf.nn.softmax(logits_2),\
                      tf.nn.softmax(logits_3),\
                      tf.nn.softmax(logits_4),\
                      tf.nn.softmax(logits_5)])
    valid_prediction = tf.pack([tf.nn.softmax(model(tf_valid_dataset)[0]),\
                      tf.nn.softmax(model(tf_valid_dataset)[1]),\
                      tf.nn.softmax(model(tf_valid_dataset)[2]),\
                      tf.nn.softmax(model(tf_valid_dataset)[3]),\
                      tf.nn.softmax(model(tf_valid_dataset)[4])])
    test_prediction = tf.pack([tf.nn.softmax(model(tf_test_dataset)[0]),\
                     tf.nn.softmax(model(tf_test_dataset)[1]),\
                     tf.nn.softmax(model(tf_test_dataset)[2]),\
                     tf.nn.softmax(model(tf_test_dataset)[3]),\
                     tf.nn.softmax(model(tf_test_dataset)[4])])

In [54]:
# def accuracy(predictions, labels):
#     return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
#           / predictions.shape[0])

In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])


In [64]:
num_steps = 20001

with tf.Session(graph=graph) as session:
#     writer = tf.train.SummaryWriter("logs/", session.graph)
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels[:,1:6]))
#             print(np.argmax(predictions, 1), np.argmax(batch_labels[:,1:6],1))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels[:,1:6]))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))

Initialized
Minibatch loss at step 0: 1079.544678
Minibatch accuracy: 1.6%
Validation accuracy: 59.2%
Minibatch loss at step 500: 5.537723
Minibatch accuracy: 66.4%
Validation accuracy: 64.5%
Minibatch loss at step 1000: 5.493402
Minibatch accuracy: 67.0%
Validation accuracy: 64.5%
Minibatch loss at step 1500: 5.615956
Minibatch accuracy: 63.6%
Validation accuracy: 64.5%
Minibatch loss at step 2000: 5.762838
Minibatch accuracy: 63.8%
Validation accuracy: 64.5%
Minibatch loss at step 2500: 5.474291
Minibatch accuracy: 66.6%
Validation accuracy: 64.5%
Minibatch loss at step 3000: 5.868134
Minibatch accuracy: 62.8%
Validation accuracy: 64.5%
Minibatch loss at step 3500: 5.542148
Minibatch accuracy: 65.2%
Validation accuracy: 64.5%
Minibatch loss at step 4000: 5.684011
Minibatch accuracy: 63.8%
Validation accuracy: 64.5%
Minibatch loss at step 4500: 5.375666
Minibatch accuracy: 65.5%
Validation accuracy: 64.5%
Minibatch loss at step 5000: 5.537566
Minibatch accuracy: 64.7%
Validation accur

In [61]:
batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden = 64

image_size = 32
num_labels = 11 # 0-9, + blank 
num_channels = 1 # grayscale

graph = tf.Graph()

with graph.as_default():

    def weight_varible(shape):
        initial = tf.truncated_normal(shape, stddev = 0.1)
        return tf.Variable(initial)

    def bias_varible(shape):
        initial = tf.constant(1.0, shape = shape)
        return tf.Variable(initial)
    
    def conv2d(data, weight):
        # strides [1, x_movement, y_movement, 1]
        return tf.nn.conv2d(data, weight, strides = [1, 1, 1, 1], padding = 'VALID')

    def max_pooling(data):
        return tf.nn.max_pool(data, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
#     def get_class_wb():
#         weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
#         biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
#         return weights, biases #, logits
    
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)
    
    
    # Varibles
    # conv1 layer 1
#     layer1_weights = weight_varible([patch_size, patch_size, num_channels, depth1])
    layer1_weights = tf.get_variable('W1',shape=[patch_size, patch_size, num_channels, depth1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = bias_varible([depth1]) # 16
    # conv2 layer 2
#     layer2_weights = weight_varible([patch_size, patch_size, depth1, depth2]) # in depth1, out depth2
    layer2_weights = tf.get_variable('W2',shape=[patch_size, patch_size, depth1, depth2],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = bias_varible([depth2]) # 32
    # conv3 layer 3
#     layer3_weights = weight_varible([patch_size, patch_size, depth2, depth3]) # in depth2, out depth3
    layer3_weights = tf.get_variable('W3',shape=[patch_size, patch_size, depth2, depth3],\
                                     initializer=tf.contrib.layers.xavier_initializer())
    layer3_biases = bias_varible([depth3]) # 64
    
    # func1 layer 4
#     layer4_weights = weight_varible([image_size // 32 * image_size // 32 * depth3, num_hidden])
#     layer4_weights = tf.get_variable('W4',shape=[image_size // 32 * image_size // 32 * depth3, num_hidden],\
#                                      initializer=tf.contrib.layers.xavier_initializer_conv2d())
#     layer4_biases = bias_varible([num_hidden])

    s1_w = tf.get_variable("WS1", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s1_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS1')
    s2_w = tf.get_variable("WS2", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s2_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS2')
    s3_w = tf.get_variable("WS3", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s3_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS3')
    s4_w = tf.get_variable("WS4", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s4_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS4')
    s5_w = tf.get_variable("WS5", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s5_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS5')
    
#     s1_w, s1_b = get_class_wb()
#     s2_w, s2_b = get_class_wb()
#     s3_w, s3_b = get_class_wb()
#     s4_w, s4_b = get_class_wb()
#     s5_w, s5_b = get_class_wb()
    sw = [s1_w, s2_w, s3_w, s4_w, s5_w]
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    def model(dataset):
        # conv1 layer 1
        hidden1 = tf.nn.relu(conv2d(dataset, layer1_weights) + layer1_biases) # 28 * 28 * depth1
#         hidden1 = tf.nn.local_response_normalization(hidden1)
        pool1 = max_pooling(hidden1) # 14 * 14 * depth1
        # conv2 layer 2
        hidden2 = tf.nn.relu(conv2d(pool1, layer2_weights) + layer2_biases) # 10 * 10 * depth2
#         hidden2 = tf.nn.local_response_normalization(hidden2)
        pool2 = max_pooling(hidden2) # 5 * 5 * depth2
        # conv3 layer 3
        pool3 = tf.nn.relu(conv2d(pool2, layer3_weights) + layer3_biases) # 1 * 1 * depth3
#         pool3 = max_pooling(hidden3) # 1 * 1 * depth3
        
        shape = pool3.get_shape().as_list()
        pool3_flat = tf.reshape(pool3, [shape[0], shape[1] * shape[2] * shape[3]])
        
#         # func1 layer 4
#         hidden4 = tf.nn.relu(tf.matmul(pool3_flat, layer4_weights) + layer4_biases)
        hidden4_drop = tf.nn.dropout(pool3_flat, 0.9375)

        logits_1 = tf.matmul(hidden4_drop, s1_w) + s1_b
        logits_2 = tf.matmul(hidden4_drop, s2_w) + s2_b
        logits_3 = tf.matmul(hidden4_drop, s3_w) + s3_b
        logits_4 = tf.matmul(hidden4_drop, s4_w) + s4_b
        logits_5 = tf.matmul(hidden4_drop, s5_w) + s5_b
        
        return [logits_1, logits_2, logits_3, logits_4, logits_5]
    
# Training computation.
#     logits = model(tf_train_dataset)
    [logits_1, logits_2, logits_3, logits_4, logits_5] = model(tf_train_dataset)
    
#     loss_per_digit = [tf.reduce_mean(
#                         tf.nn.sparse_softmax_cross_entropy_with_logits(
#                             logits[i],
#                             tf_train_labels[:,i+1]
#                         )) + beta_regul * tf.nn.l2_loss(sw[i])
#                        for i in range(5)]
    
#     loss = tf.add_n(loss_per_digit)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_1, tf_train_labels[:,1])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_2, tf_train_labels[:,2])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_3, tf_train_labels[:,3])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_4, tf_train_labels[:,4])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_5, tf_train_labels[:,5]))


    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.70, staircase=True)
    optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.pack([tf.nn.softmax(model(tf_train_dataset)[0]),\
                      tf.nn.softmax(model(tf_train_dataset)[1]),\
                      tf.nn.softmax(model(tf_train_dataset)[2]),\
                      tf.nn.softmax(model(tf_train_dataset)[3]),\
                      tf.nn.softmax(model(tf_train_dataset)[4])])
#     train_prediction = tf.pack([tf.nn.softmax(logits[i]) for i in range(5)])
    valid_prediction = tf.pack([tf.nn.softmax(model(tf_valid_dataset)[0]),\
                      tf.nn.softmax(model(tf_valid_dataset)[1]),\
                      tf.nn.softmax(model(tf_valid_dataset)[2]),\
                      tf.nn.softmax(model(tf_valid_dataset)[3]),\
                      tf.nn.softmax(model(tf_valid_dataset)[4])])
    test_prediction = tf.pack([tf.nn.softmax(model(tf_test_dataset)[0]),\
                     tf.nn.softmax(model(tf_test_dataset)[1]),\
                     tf.nn.softmax(model(tf_test_dataset)[2]),\
                     tf.nn.softmax(model(tf_test_dataset)[3]),\
                     tf.nn.softmax(model(tf_test_dataset)[4])])
    

In [62]:
num_steps = 20001

with tf.Session(graph=graph) as session:
#     writer = tf.train.SummaryWriter("logs/", session.graph)
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels[:,1:6]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels[:,1:6]))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))

Initialized
Minibatch loss at step 0: 21.555344
Minibatch accuracy: 9.7%
Validation accuracy: 55.7%
Minibatch loss at step 500: 5.558725
Minibatch accuracy: 61.9%
Validation accuracy: 65.6%
Minibatch loss at step 1000: 4.721919
Minibatch accuracy: 68.8%
Validation accuracy: 70.8%
Minibatch loss at step 1500: 3.649049
Minibatch accuracy: 78.8%
Validation accuracy: 73.8%
Minibatch loss at step 2000: 4.099685
Minibatch accuracy: 75.3%
Validation accuracy: 75.9%
Minibatch loss at step 2500: 3.093337
Minibatch accuracy: 80.3%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 3.714985
Minibatch accuracy: 78.1%
Validation accuracy: 77.0%
Minibatch loss at step 3500: 3.654895
Minibatch accuracy: 75.6%
Validation accuracy: 77.4%
Minibatch loss at step 4000: 2.732865
Minibatch accuracy: 85.3%
Validation accuracy: 77.7%
Minibatch loss at step 4500: 3.464983
Minibatch accuracy: 79.1%
Validation accuracy: 78.0%
Minibatch loss at step 5000: 3.847943
Minibatch accuracy: 77.8%
Validation accurac

In [63]:
print(np.argmax(predictions, 2).T)

[[ 1  1 10 10 10]
 [ 6 10 10 10 10]
 [ 3 10 10 10 10]
 [ 2  5  1 10 10]
 [ 5 10 10 10 10]
 [ 6  2 10 10 10]
 [ 4  5 10 10 10]
 [ 5  1 10 10 10]
 [ 2  7  4 10 10]
 [ 3 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1  6  7 10 10]
 [ 4 10 10 10 10]
 [ 1  5 10 10 10]
 [ 2 10 10 10 10]
 [ 4 10 10 10 10]
 [ 8  2 10 10 10]
 [ 1  4 10 10 10]
 [ 6 10 10 10 10]
 [ 1  5 10 10 10]
 [ 1  8  8 10 10]
 [ 5 10 10 10 10]
 [ 1  5 10 10 10]
 [ 1  3 10 10 10]
 [ 5  2  3 10 10]
 [ 3  2 10 10 10]
 [ 1  7 10 10 10]
 [ 1 10 10 10 10]
 [ 5 10 10 10 10]
 [ 2 10 10 10 10]
 [ 1  4 10 10 10]
 [ 3  2 10 10 10]
 [ 1  6 10 10 10]
 [ 2 10 10 10 10]
 [ 2  5  7 10 10]
 [ 5 10 10 10 10]
 [ 2 10 10 10 10]
 [ 4  3 10 10 10]
 [ 5 10 10 10 10]
 [ 1  6 10 10 10]
 [ 7  1 10 10 10]
 [ 9  9 10 10 10]
 [ 1  3 10 10 10]
 [ 7 10 10 10 10]
 [ 1 10 10 10 10]
 [ 3  9 10 10 10]
 [ 5  3 10 10 10]
 [ 1 10 10 10 10]
 [ 1  8 10 10 10]
 [ 6 10 10 10 10]
 [ 3  9 10 10 10]
 [ 4  5 10 10 10]
 [ 1  9 10 10 10]
 [ 6  3 10 10 10]
 [ 3  2 10 10 10]
 [ 1 10 10

In [12]:
batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden = 1024

image_size = 32
num_labels = 11 # 0-9, + blank 
num_channels = 1 # grayscale

graph = tf.Graph()

with graph.as_default():

    def weight_varible(shape):
        initial = tf.truncated_normal(shape, stddev = 0.1)
        return tf.Variable(initial)

    def bias_varible(shape):
        initial = tf.constant(1.0, shape = shape)
        return tf.Variable(initial)
    
    def conv2d(data, weight):
        # strides [1, x_movement, y_movement, 1]
        return tf.nn.conv2d(data, weight, strides = [1, 1, 1, 1], padding = 'SAME')

    def max_pooling(data):
        return tf.nn.max_pool(data, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
#     def get_class_wb():
#         weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
#         biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
#         return weights, biases #, logits
    
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta_regul = tf.placeholder(tf.float32)
    
    
    # Varibles
    # conv1 layer 1
#     layer1_weights = weight_varible([patch_size, patch_size, num_channels, depth1])
    layer1_weights = tf.get_variable('W1',shape=[patch_size, patch_size, num_channels, depth1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = bias_varible([depth1]) # 16
    # conv2 layer 2
#     layer2_weights = weight_varible([patch_size, patch_size, depth1, depth2]) # in depth1, out depth2
    layer2_weights = tf.get_variable('W2',shape=[patch_size, patch_size, depth1, depth2],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = bias_varible([depth2]) # 32
    # conv3 layer 3
#     layer3_weights = weight_varible([patch_size, patch_size, depth2, depth3]) # in depth2, out depth3
    layer3_weights = tf.get_variable('W3',shape=[patch_size, patch_size, depth2, depth3],\
                                     initializer=tf.contrib.layers.xavier_initializer())
    layer3_biases = bias_varible([depth3]) # 64
    
    # func1 layer 4
#     layer4_weights = weight_varible([image_size // 32 * image_size // 32 * depth3, num_hidden])
    layer4_weights = tf.get_variable('W4',shape=[image_size // 8 * image_size // 8 * depth3, num_hidden],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer4_biases = bias_varible([num_hidden])

    s1_w = tf.get_variable("WS1", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s1_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS1')
    s2_w = tf.get_variable("WS2", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s2_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS2')
    s3_w = tf.get_variable("WS3", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s3_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS3')
    s4_w = tf.get_variable("WS4", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s4_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS4')
    s5_w = tf.get_variable("WS5", shape=[num_hidden, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    s5_b = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='BS5')
    
    sw = [s1_w, s2_w, s3_w, s4_w, s5_w]
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    
    def model(dataset, keep_prob):
        # conv1 layer 1
        hidden1 = tf.nn.relu(conv2d(dataset, layer1_weights) + layer1_biases) # 28 * 28 * depth1
        pool1 = max_pooling(hidden1) # 14 * 14 * depth1
        # conv2 layer 2
        hidden2 = tf.nn.relu(conv2d(pool1, layer2_weights) + layer2_biases) # 10 * 10 * depth2
        pool2 = max_pooling(hidden2) # 5 * 5 * depth2
        # conv3 layer 3
        hidden3 = tf.nn.relu(conv2d(pool2, layer3_weights) + layer3_biases) # 1 * 1 * depth3
        pool3 = max_pooling(hidden3) # 1 * 1 * depth3
        
        shape = pool3.get_shape().as_list()
        pool3_flat = tf.reshape(pool3, [shape[0], shape[1] * shape[2] * shape[3]])
        
#         # func1 layer 4
        hidden4 = tf.nn.relu(tf.matmul(pool3_flat, layer4_weights) + layer4_biases)
        hidden4_drop = tf.nn.dropout(hidden4, keep_prob)

        logits_1 = tf.matmul(hidden4_drop, s1_w) + s1_b
        logits_2 = tf.matmul(hidden4_drop, s2_w) + s2_b
        logits_3 = tf.matmul(hidden4_drop, s3_w) + s3_b
        logits_4 = tf.matmul(hidden4_drop, s4_w) + s4_b
        logits_5 = tf.matmul(hidden4_drop, s5_w) + s5_b
        
        return [logits_1, logits_2, logits_3, logits_4, logits_5]
    
# Training computation.
#     logits = model(tf_train_dataset)
    [logits_1, logits_2, logits_3, logits_4, logits_5] = model(tf_train_dataset, 0.9375)
    
#     loss_per_digit = [tf.reduce_mean(
#                         tf.nn.sparse_softmax_cross_entropy_with_logits(
#                             logits[i],
#                             tf_train_labels[:,i+1]
#                         )) + beta_regul * tf.nn.l2_loss(sw[i])
#                        for i in range(5)]
    
#     loss = tf.add_n(loss_per_digit)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_1, tf_train_labels[:,1])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_2, tf_train_labels[:,2])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_3, tf_train_labels[:,3])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_4, tf_train_labels[:,4])) +\
tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_5, tf_train_labels[:,5]))


    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.70, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.pack([tf.nn.softmax(model(tf_train_dataset, 1.0)[0]),\
                      tf.nn.softmax(model(tf_train_dataset, 1.0)[1]),\
                      tf.nn.softmax(model(tf_train_dataset, 1.0)[2]),\
                      tf.nn.softmax(model(tf_train_dataset, 1.0)[3]),\
                      tf.nn.softmax(model(tf_train_dataset, 1.0)[4])])
#     train_prediction = tf.pack([tf.nn.softmax(logits[i]) for i in range(5)])
    valid_prediction = tf.pack([tf.nn.softmax(model(tf_valid_dataset, 1.0)[0]),\
                      tf.nn.softmax(model(tf_valid_dataset, 1.0)[1]),\
                      tf.nn.softmax(model(tf_valid_dataset, 1.0)[2]),\
                      tf.nn.softmax(model(tf_valid_dataset, 1.0)[3]),\
                      tf.nn.softmax(model(tf_valid_dataset, 1.0)[4])])
    test_prediction = tf.pack([tf.nn.softmax(model(tf_test_dataset, 1.0)[0]),\
                     tf.nn.softmax(model(tf_test_dataset, 1.0)[1]),\
                     tf.nn.softmax(model(tf_test_dataset, 1.0)[2]),\
                     tf.nn.softmax(model(tf_test_dataset, 1.0)[3]),\
                     tf.nn.softmax(model(tf_test_dataset, 1.0)[4])])
    

In [13]:
num_steps = 20001

with tf.Session(graph=graph) as session:
#     writer = tf.train.SummaryWriter("logs/", session.graph)
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels[:,1:6]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels[:,1:6]))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:6]))

Initialized
Minibatch loss at step 0: 24.621223
Minibatch accuracy: 1.6%
Validation accuracy: 62.7%
Minibatch loss at step 500: 6.172132
Minibatch accuracy: 61.6%
Validation accuracy: 61.1%
Minibatch loss at step 1000: 5.791682
Minibatch accuracy: 65.3%
Validation accuracy: 64.5%
Minibatch loss at step 1500: 5.518498
Minibatch accuracy: 67.2%
Validation accuracy: 64.5%


KeyboardInterrupt: 

In [16]:
print(batch_labels[:,1:6])

[[ 2  6 10 10 10]
 [ 1  4 10 10 10]
 [ 1  3 10 10 10]
 [ 1  1  1 10 10]
 [ 1  1  2 10 10]
 [ 6 10 10 10 10]
 [ 4 10 10 10 10]
 [ 3  2  7 10 10]
 [ 3  3 10 10 10]
 [ 1  3  8 10 10]
 [ 1  8 10 10 10]
 [ 2 10 10 10 10]
 [ 1  9  6 10 10]
 [ 2  1 10 10 10]
 [ 3  2 10 10 10]
 [ 3  7  2 10 10]
 [ 4  3 10 10 10]
 [ 1  5  6 10 10]
 [ 2  2 10 10 10]
 [ 2  9 10 10 10]
 [ 1  2  5 10 10]
 [ 6 10 10 10 10]
 [ 3  5 10 10 10]
 [ 4  3 10 10 10]
 [ 5  9  4 10 10]
 [ 1  3  6 10 10]
 [ 1  1 10 10 10]
 [ 1  2  5  7 10]
 [ 9  3 10 10 10]
 [ 4  2 10 10 10]
 [ 4  6 10 10 10]
 [ 5 10 10 10 10]
 [ 4  3 10 10 10]
 [ 2  7 10 10 10]
 [ 1 10 10 10 10]
 [ 2  5  7 10 10]
 [ 7 10 10 10 10]
 [ 4  4 10 10 10]
 [ 5  1 10 10 10]
 [ 1 10 10 10 10]
 [ 2  7 10 10 10]
 [ 5  8 10 10 10]
 [ 1  6  3 10 10]
 [ 4  1 10 10 10]
 [ 1  5  3 10 10]
 [ 1  2  4 10 10]
 [ 3 10  6 10 10]
 [ 2  2  9 10 10]
 [ 6  8 10 10 10]
 [ 3 10 10 10 10]
 [ 1  4 10 10 10]
 [ 4  6 10 10 10]
 [ 5  3 10 10 10]
 [ 5 10  2 10 10]
 [ 6  3  7 10 10]
 [ 2  4  8

In [14]:
print(np.argmax(predictions, 2).T)

[[ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10 10 10]
 [ 1 10 10